In [124]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:

import os
import sys
sys.path.append('src/')
# scripts
from clean_helpers import *
#libraries
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# modeling
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [134]:
train,X_test = pd.read_csv('data/train.csv'),pd.read_csv('data/test.csv')

In [135]:
train.Age=train.Age.fillna(value=train.Age.mean())
train.Embarked=train.Embarked.fillna(value='Missing')
train.Embarked=train.Embarked.replace('C','Cherbourg').replace('Q','Queenstown').replace('S','Southampton')
col_list=['PassengerId','Cabin','Ticket','Name']
for col in col_list:
    train.drop(col,axis=1,inplace=True)
train = train.dropna()

y_train = train.Survived
X_train = train.drop('Survived',axis=1)

ohe = OneHotEncoder()
cols = ['Pclass','Sex','Parch','SibSp','Embarked']
for col in cols:
    feature = np.array(X_train[col]).reshape(-1,1)
    ohe.fit(feature)
    encoded = pd.DataFrame(ohe.transform(feature).toarray())
    X_train = pd.concat([X_train,encoded],axis=1)
    for name in encoded.columns:
        X_train.rename(columns={name:f'{col}: {name}'},inplace=True)
    X_train.drop(col,inplace=True,axis=1)
    
X_train.head()

,Age,Fare,Pclass: 0,Pclass: 1,Pclass: 2,Sex: 0,Sex: 1,Parch: 0,Parch: 1,Parch: 2,...,SibSp: 1,SibSp: 2,SibSp: 3,SibSp: 4,SibSp: 5,SibSp: 6,Embarked: 0,Embarked: 1,Embarked: 2,Embarked: 3
0,22.0,7.2500,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,38.0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,26.0,7.9250,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,35.0,53.1000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,35.0,8.0500,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [136]:
X_test.Age=X_test.Age.fillna(value=X_test.Age.mean())
X_test.Embarked=X_test.Embarked.fillna(value='Missing')
X_test.Embarked=X_test.Embarked.replace('C','Cherbourg').replace('Q','Queenstown').replace('S','Southampton')
col_list=['PassengerId','Cabin','Ticket','Name']
for col in col_list:
    X_test.drop(col,axis=1,inplace=True)
X_test = X_test.dropna()

ohe = OneHotEncoder()
cols = ['Pclass','Sex','Parch','SibSp','Embarked']
for col in cols:
    feature = np.array(X_test[col]).reshape(-1,1)
    ohe.fit(feature)
    encoded = pd.DataFrame(ohe.transform(feature).toarray())
    X_test = pd.concat([X_test,encoded],axis=1)
    for name in encoded.columns:
        X_test.rename(columns={name:f'{col}: {name}'},inplace=True)
        X_test.dropna(inplace=True)
    X_test.drop(col,inplace=True,axis=1)

In [ ]:
X_train, X_test, y_train, y_test = 